In [10]:
%%writefile mapper.py
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
stop_words = ['a','about','above','across','after','afterwards','again','against','all','almost','alone','along','already','also','although','always','am','among','amongst','amoungst','amount','an','and','another','any','anyhow','anyone','anything','anyway','anywhere','are','around','as','at','back','be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','below','beside','besides','between','beyond','bill','both','bottom','but','by','call','can','cannot','cant','co','computer','con','could','couldnt','cry','de','describe','detail','do','done','down','due','during','each','eg','eight','either','eleven','else','elsewhere','empty','enough','etc','even','ever','every','everyone','everything','everywhere','except','few','fifteen','fify','fill','find','fire','first','five','for','former','formerly','forty','found','four','from','front','full','further','get','give','go','had','has','hasnt','have','he','hence','her','here','hereafter','hereby','herein','hereupon','hers','herse\"','him','himse\"','his','how','however','hundred','i','ie','if','in','inc','indeed','interest','into','is','it','its','itse\"','keep','last','latter','latterly','least','less','ltd','made','many','may','me','meanwhile','might','mill','mine','more','moreover','most','mostly','move','much','must','my','myse\"','name','namely','neither','never','nevertheless','next','nine','no','nobody','none','noone','nor','not','nothing','now','nowhere','of','off','often','on','once','one','only','onto','or','other','others','otherwise','our','ours','ourselves','out','over','own','part','per','perhaps','please','put','rather','re','same','see','seem','seemed','seeming','seems','serious','several','she','should','show','side','since','sincere','six','sixty','so','some','somehow','someone','something','sometime','sometimes','somewhere','still','such','system','take','ten','than','that','the','their','them','themselves','then','thence','there','thereafter','thereby','therefore','therein','thereupon','these','they','thick','thin','third','this','those','though','three','through','throughout','thru','thus','to','together','too','top','toward','towards','twelve','twenty','two','un','under','until','up','upon','us','very','via','was','we','well','were','what','whatever','when','whence','whenever','where','whereafter','whereas','whereby','wherein','whereupon','wherever','whether','which','while','whither','who','whoever','whole','whom','whose','why','will','with','within','without','would','yet','you','your','yours','yourself','yourselves']
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        print(e)
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        word = word.lower()
        if word not in stop_words and (word == 'labor' or int(article_id) == 12):
            key = word + "," + article_id
            print "%s\t%d" % (key, 1)

Overwriting mapper.py


In [11]:
%%writefile reducer.py
import sys
current_article_id = None
current_word = None
word_sum = 0
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        word, article_id = key.split(',', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_article_id != article_id or current_word != word:
        if current_article_id and current_word:
            key = current_word + "," + current_article_id
            _value = str(word_sum) + "," + str(6092)
            print "%s\t%s" % (key, _value)
            word_sum = 0
        current_article_id = article_id
        current_word = word
    word_sum += count
key = current_word + "," + current_article_id
_value = str(word_sum) + "," + str(6092)
print "%s\t%s" % (key, _value)

Overwriting reducer.py


In [12]:
%%writefile mapper3.py
import sys
for line in sys.stdin:
    key, value = line.strip().split('\t', 1)
    word, article_id = key.split(',', 1)
    word_sum, words_in_article = value.split(',', 1)
    value = article_id + "," + word_sum + "," + words_in_article + "," + str(1)
    print '%s\t%s' % (word, value)

Overwriting mapper3.py


In [13]:
%%writefile reducer3.py
import sys
import math
current_word = None
word_in_articles_count = 0
word_stats_dict = {}
word_in_articles_counts = {}
for line in sys.stdin:
    try:
      word, value = line.strip().split('\t', 1)
      word_stats = value.split(',')
      count = int(word_stats[3])
    except ValueError as e:
      continue
    if current_word != word:
        if current_word:
            word_in_articles_counts[current_word] = word_in_articles_count
            word_in_articles_count = 0
        current_word = word
    word_in_articles_count += int(count)
    
    if word in word_stats_dict.keys():
        word_stats_dict[word].append(word_stats[0] + "," + word_stats[1] + "," + word_stats[2])
    else:
        word_stats_dict[word] = []
        word_stats_dict[word].append(word_stats[0] + "," + word_stats[1] + "," + word_stats[2])
word_in_articles_counts[current_word] = word_in_articles_count
for word in word_in_articles_counts.keys():
    word_stats_list = word_stats_dict[word]
    for word_stats in word_stats_list:
        word_stats = word_stats.split(",")
        try :
          key = word + "," + word_stats[0]
          print "%s\t%f" % (key, (float(word_stats[1])/float(word_stats[2]))*(1/math.log(1+word_in_articles_counts[word])))  
        except ZeroDivisionError as e:
          continue

Overwriting reducer3.py


In [ ]:
%%bash
NUM_REDUCERS=8
OUT_DIR="tfidf_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_FINAL_DIR="tfidf_final_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_FINAL_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=1 \
    -files mapper3.py,reducer3.py \
    -mapper "python mapper3.py" \
    -reducer "python reducer3.py" \
    -input ${OUT_DIR} \
    -output ${OUT_FINAL_DIR} > /dev/null
hdfs dfs -cat ${OUT_FINAL_DIR}/part-00000 | awk -F '\t' '$1 == "labor,12" {print $2}'